# DENSE-Inception U-net for medical image segmentation

### [DOI](https://doi.org/10.1016/j.cmpb.2020.105395)

In [0]:
!pip install segmentation-models
# Import Libraries
import os
import segmentation_models as sm
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
import keras
!pip install segmentation_models
!pip install split-folders
!pip install nibabel
#--------------------------------
import time, shutil
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
from skimage.transform import resize
from skimage.io import imread, imsave
#--------------------------------
import nibabel as nib
import segmentation_models as sm
import split_folders as sf
#--------------------------------
from DataGenerator3 import DataGenerator

## Inception-Res Block Drawing

![InceptRes](https://i.imgur.com/HyeWqty.jpg)

## Inception Res Block Implementation

In [0]:
def InceptionResBlock(block_input, num_filters):

    x1 = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(block_input)
    x1 = keras.layers.BatchNormalization(epsilon=1e-06,  momentum=0.9, weights=None)(x1)
    
    x2 = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(block_input)

    # x2 = keras.layers.Conv2D(num_filters,(3,3),activation='relu',
    #                          kernel_initializer='he_uniform', padding='same')(x2)
    x2 = keras.layers.Conv2D(num_filters,(1,3),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(x2)
    x2 = keras.layers.Conv2D(num_filters,(3,1),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(x2)

    x2 = keras.layers.BatchNormalization(epsilon=1e-06,  momentum=0.9, weights=None)(x2)
    
    x3 = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(block_input)

    # x3 = keras.layers.Conv2D(num_filters,(3,3),activation='relu',
    #                          kernel_initializer='he_uniform', padding='same')(x3)
    x3 = keras.layers.Conv2D(num_filters,(3,1),activation='relu',
                            kernel_initializer='he_uniform', padding='same')(x3)
    x3 = keras.layers.Conv2D(num_filters,(1,3),activation='relu',
                            kernel_initializer='he_uniform', padding='same')(x3)

    x3 = keras.layers.BatchNormalization(epsilon=1e-06,  momentum=0.9, weights=None)(x3)

    # x3 = keras.layers.Conv2D(num_filters,(3,3),activation='relu',
    #                          kernel_initializer='he_uniform', padding='same')(x3)
    x3 = keras.layers.Conv2D(num_filters,(3,1),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(x3)
    x3 = keras.layers.Conv2D(num_filters,(1,3),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(x3)

    x3 = keras.layers.BatchNormalization(epsilon=1e-06,  momentum=0.9, weights=None)(x3)
    # print('Incept-Res x1: ',x1.shape,'x2: ',x2.shape,'x3: ',x3.shape)
    x = keras.layers.Concatenate(axis=-1)([x1, x2, x3])
    # x = x1 + x2 + x3
    # Bottleneck Layer
    x = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
                            kernel_initializer='he_uniform', padding='same')(x) 
    #================
    ## Modification 
    #================
    print('INceptionRes: \n input: ',block_input.shape,'Output:', x.shape,'\n-----------------------------\n')
    block_input_modified = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
                                               padding='same')(block_input)
    out = keras.layers.Add()([x, block_input_modified])
    #out = keras.layers.Add()([x, block_input])

    return out

## Inception-Res in Dense Block  Drawing

![Incept Res Dense](https://i.imgur.com/VSaf4x8.jpg)

## Inception Res Dense Block Implementation

In [0]:
def InceptionResDenseBlock(block_input, num_filters):

    x1 = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(block_input)
    x1 = keras.layers.BatchNormalization(epsilon=1e-06,  momentum=0.9, weights=None)(x1)
    
    # x2 = keras.layers.MaxPool2D(pool_size=(3,3) ,strides=(1,1),padding='same')(block_input) 
    x2 = keras.layers.MaxPool2D(pool_size=(3,1) ,strides=(1,1),padding='same')(block_input)
    x2 = keras.layers.MaxPool2D(pool_size=(1,3) ,strides=(1,1),padding='same')(x2)  

    x2 = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(x2)
    x2 = keras.layers.BatchNormalization(epsilon=1e-06,  momentum=0.9, weights=None)(x2)
    
    x3 = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(block_input)
    x3 = keras.layers.Conv2D(num_filters,(1,3),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(x3)
    x3 = keras.layers.BatchNormalization(epsilon=1e-06,  momentum=0.9, weights=None)(x3)
    x3 = keras.layers.Conv2D(num_filters,(3,1),activation='relu',
                             kernel_initializer='he_uniform', padding='same')(x3)
    x3 = keras.layers.BatchNormalization(epsilon=1e-06,  momentum=0.9, weights=None)(x3)

    # print('Incept_Dense_Res x1: ',x1.shape,'x2: ',x2.shape,'x3: ',x3.shape)
    x = keras.layers.Concatenate(axis=-1)([x1, x2, x3])
    # x = x1 + x2 + x3
    # Bottleneck Layer
    x = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
                            kernel_initializer='he_uniform', padding='same')(x)   
    #================
    ## Modification 
    #================
    block_input_modified = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
                                               padding='same')(block_input)
    # print('INceptionResDesnse: \n input: ',block_input.shape,'Output:', x.shape,'\n-----------------------------\n')
    out = keras.layers.Add()([x, block_input_modified])
    # out = keras.layers.Add()([x, block_input])

    return out

## Down & Up Sampling  Drawing

![ Down & Up Sampling ](https://i.imgur.com/ntWshKK.jpg)

## DownSampling Block Implementation

In [0]:
def DownSamplingBlock(block_input, num_filters):
    
    x1 = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
        kernel_initializer='he_uniform', padding='same')(block_input)
    # x1 = keras.layers.Conv2D(num_filters,(3,3),activation='relu',
    #     kernel_initializer='he_uniform', padding='same')(x1)
    x1 = keras.layers.Conv2D(num_filters,(3,1),activation='relu',
        kernel_initializer='he_uniform', padding='same')(x1)
    x1 = keras.layers.Conv2D(num_filters,(1,3),activation='relu',
        kernel_initializer='he_uniform', padding='same')(x1)
    # x1 = keras.layers.Conv2D(num_filters,(3,3), strides=(2,2),
    #     activation='relu',kernel_initializer='he_uniform', padding='same')(x1)
    x1 = keras.layers.Conv2D(num_filters,(3,1), strides=(2,1),
        activation='relu',kernel_initializer='he_uniform', padding='same')(x1)
    x1 = keras.layers.Conv2D(num_filters,(1,3), strides=(1,2),
        activation='relu',kernel_initializer='he_uniform', padding='same')(x1)

    x2 = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
        kernel_initializer='he_uniform', padding='same')(block_input)
    # x2 = keras.layers.Conv2D(num_filters,(3,3), strides=(2,2),
    #     activation='relu',kernel_initializer='he_uniform', padding='same')(x2)
    x2 = keras.layers.Conv2D(num_filters,(3,1), strides=(2,1),
        activation='relu',kernel_initializer='he_uniform', padding='same')(x2)
    x2 = keras.layers.Conv2D(num_filters,(1,3), strides=(1,2),
        activation='relu',kernel_initializer='he_uniform', padding='same')(x2)

    x3 = keras.layers.MaxPool2D(strides=(2,2))(block_input)
    
    # print('Down x1: ',x1.shape,'x2: ',x2.shape,'x3: ',x3.shape)
    x = keras.layers.Concatenate(axis=-1)([x1, x2, x3])
    # x = x1 + x2 + x3
    # Bottleneck Layer
    out = keras.layers.Conv2D(num_filters,(1,1),activation='relu',
        kernel_initializer='he_uniform', padding='same')(x) 
    #================
    ## Modification 
    #================ 
    print('DownSample: \n input: ',block_input.shape,'Output:',out.shape, '\n-----------------------------\n')
    return out

## UpSampling BLock Implementation

In [0]:
def UpSamplingBlock(block_input, num_filters):
    
    # x1 = keras.layers.Conv2DTranspose(num_filters, (3,3),padding='same')(block_input)
    x1 = keras.layers.Conv2DTranspose(num_filters, (3,1),padding='same')(block_input)
    x1 = keras.layers.Conv2DTranspose(num_filters, (1,3),padding='same')(x1)
    # x1 = keras.layers.Conv2DTranspose(num_filters, (3,3),padding='same')(x1)
    x1 = keras.layers.Conv2DTranspose(num_filters, (1,3),padding='same')(x1)
    x1 = keras.layers.Conv2DTranspose(num_filters, (3,1),padding='same')(x1)
    # x1 = keras.layers.Conv2DTranspose(num_filters, (3,3), strides=(2,2),padding='same')(x1)
    x1 = keras.layers.Conv2DTranspose(num_filters, (3,1), strides=(2,1),padding='same')(x1)
    x1 = keras.layers.Conv2DTranspose(num_filters, (1,3), strides=(1,2),padding='same')(x1)

    # x2 = keras.layers.Conv2DTranspose(num_filters, (3,3),padding='same')(block_input)
    x2 = keras.layers.Conv2DTranspose(num_filters, (1,3),padding='same')(block_input)
    x2 = keras.layers.Conv2DTranspose(num_filters, (3,1),padding='same')(x2)
    # x2 = keras.layers.Conv2DTranspose(num_filters, (3,3), strides=(2,2),padding='same')(x2)
    x2 = keras.layers.Conv2DTranspose(num_filters, (3,1), strides=(2,1),padding='same')(x2)
    x2 = keras.layers.Conv2DTranspose(num_filters, (1,3), strides=(1,2),padding='same')(x2)

    x3 = keras.layers.UpSampling2D((2,2))(block_input)
    x3 = keras.layers.Conv2D(num_filters, (1,1))(x3)

    # print('Up x1: ',x1.shape,'x2: ',x2.shape,'x3: ',x3.shape)
    x = keras.layers.Concatenate(axis=-1)([x1, x2, x3])
    # x = x1 + x2 + x3
    
    # Bottleneck Layer
    # out = keras.layers.Conv2D(num_filters,(3,3),activation='relu',
    #     kernel_initializer='he_uniform', padding='same')(x) 
    x = keras.layers.Conv2D(num_filters,(1,3),activation='relu',
        kernel_initializer='he_uniform', padding='same')(x)
    out = keras.layers.Conv2D(num_filters,(3,1),activation='relu',
        kernel_initializer='he_uniform', padding='same')(x)
    #================
    ## Modification 
    #================ 
    print('UpSample: \n input: ',block_input.shape,'Output:', out.shape,'\n-----------------------------\n')
    return out

## DENSE Inception 4 Blocks  Drawing

![DENSE Incept 4 Blocks](https://i.imgur.com/i0VVFe4.jpg)

## Dense Inception 4 Blocks Implementation

In [0]:
def DenseIncepton4Blocks(block_input, num_filters):
    
    x1 = InceptionResDenseBlock(block_input, num_filters)
    BL1= keras.layers.Conv2D(num_filters,(1,1),activation='relu',
        kernel_initializer='he_uniform', padding='same')(x1)

    x2 = InceptionResDenseBlock(BL1, num_filters)
    x12= keras.layers.Concatenate()([x1, x2]) 
    BL2= keras.layers.Conv2D(num_filters,(1,1),activation='relu',
        kernel_initializer='he_uniform', padding='same')(x12)

    x3 = InceptionResDenseBlock(BL2, num_filters)
    x123= keras.layers.Concatenate()([x1, x2, x3])
    BL3= keras.layers.Conv2D(num_filters,(1,1),activation='relu',
        kernel_initializer='he_uniform', padding='same')(x123)

    x4 = InceptionResDenseBlock(BL3, num_filters)
    x1234= keras.layers.Concatenate()([x1, x2, x3, x4])
    BL4= keras.layers.Conv2D(num_filters,(1,1),activation='relu',
        kernel_initializer='he_uniform', padding='same')(x1234)
    out = BL4
    #================
    ## Modification 
    #================
    print('Dense4Block: \n input: ',block_input.shape,'Output:', out.shape,'\n-----------------------------\n')

    return out


## DUI Net Drawing

![UNet](https://i.imgur.com/HN734X8.jpg)

# DUI-Net Implementation

In [0]:
def DIUNet(input_width=256, input_height=256, num_Channels=1, num_Classes=4,
 loss='binary_crossentropy',optimizer='adam',metrics = ['accuracy']):

    assert input_height % 32 == 0
    assert input_width  % 32 == 0

    inputs = keras.layers.Input((input_width, input_height,num_Channels))
    # Encoder Path:
    print('Encoder Path')
    x1 = InceptionResBlock(inputs, 64)
    d1 = DownSamplingBlock(x1, 64)

    x2 = InceptionResBlock(d1, 128)
    d2 = DownSamplingBlock(x2, 128)

    x3 = InceptionResBlock(d2, 256)
    d3 = DownSamplingBlock(x3, 256)

    # x4 = InceptionResBlock(d3, 512)
    x4 = DenseIncepton4Blocks(d3, 512)
    d4 = DownSamplingBlock(x4, 512)

    # Bottom Layer
    print('Bottom Layer')
    x5 = DenseIncepton4Blocks(d4, 1024)
    
    # Decoder Path
    print('Decoder Path')
    u1 = UpSamplingBlock(x5, 512)
    c1 = keras.layers.Concatenate()([u1,x4])
    u1 = DenseIncepton4Blocks(c1, 512)


    u2 = UpSamplingBlock(u1, 256)
    c2 = keras.layers.Concatenate()([u2,x3])
    u2 = InceptionResBlock(c2, 256)
    
    u3 = UpSamplingBlock(u2, 128)
    c3 = keras.layers.Concatenate()([u3,x2])
    u3 = InceptionResBlock(c3, 128)
    
    u4 = UpSamplingBlock(u3, 64)
    c4 = keras.layers.Concatenate()([u4,x1])
    u5 = InceptionResBlock(c4, 32)
    u5 = InceptionResBlock(u5, 1)

    # I Added This Layer !
    outputs = keras.layers.Conv2D(4,(1,1),activation='softmax', kernel_initializer='he_uniform', padding='same')(u5)
    
    print('Output Shape: ',outputs.shape)
    print('Model')

    model = keras.Model(inputs=[inputs],outputs=[outputs])

    # Compile The Model
    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    return model

### Hyber Parameters

In [0]:
loss = sm.losses.bce_dice_loss
learning_rate = 1e-3
epochs = 120
decay_rate = learning_rate *(0.95**epochs) 
# optim = keras.optimizers.RMSprop(lr=1e-3, rho=0.9, epsilon=1e-09, decay=0.0)
optim = keras.optimizers.Adam(lr = 1e-3, beta_1=0.9, beta_2=0.999, decay=decay_rate)

metrics = [sm.metrics.iou_score, sm.metrics.f1_score]

if(os.path.isfile('DIUNet_Weights.h5')):
    # Load Pretrained Model
    model = keras.models.load_model('DIUNet_Weights.h5',compile=False)
    # Compile The PreTrained Model
    model.compile(loss=loss, optimizer=optim, metrics=metrics)

model = DIUNet(loss= loss ,optimizer= optim ,metrics = metrics)
model.summary()

Encoder Path
INceptionRes: 
 input:  (None, 256, 256, 1) Output: (None, 256, 256, 64) 
-----------------------------

DownSample: 
 input:  (None, 256, 256, 64) Output: (None, 128, 128, 64) 
-----------------------------

INceptionRes: 
 input:  (None, 128, 128, 64) Output: (None, 128, 128, 128) 
-----------------------------

DownSample: 
 input:  (None, 128, 128, 128) Output: (None, 64, 64, 128) 
-----------------------------

INceptionRes: 
 input:  (None, 64, 64, 128) Output: (None, 64, 64, 256) 
-----------------------------

DownSample: 
 input:  (None, 64, 64, 256) Output: (None, 32, 32, 256) 
-----------------------------

DownSample: 
 input:  (None, 32, 32, 512) Output: (None, 16, 16, 512) 
-----------------------------

Bottom Layer
Decoder Path
UpSample: 
 input:  (None, 16, 16, 1024) Output: (None, 32, 32, 512) 
-----------------------------

UpSample: 
 input:  (None, 32, 32, 512) Output: (None, 64, 64, 256) 
-----------------------------

INceptionRes: 
 input:  (None, 6

## Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at drive


## PNG PreProcessing

In [0]:
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 1
def imgNorm(img):
  div = np.max(img) - np.min(img)

  return 255*(img - np.min(img))/div


def oneHotMask(mask, cataNum=4):
  shape = mask.shape
  shape = list(shape)
  shape.append(cataNum)
  result = np.zeros(shape)
  # print(np.max(mask))
  for i in range(cataNum):
    temp = np.zeros(mask.shape)
    temp[mask==i] = 1
    result[:,:,i] = temp
  return result

def pre_processing(imgPath, gtPath, opPath, normalize=True, oneHot=True, newSize=[IMG_WIDTH,IMG_HEIGHT], cataNum=4):
  """
  the ouput png file will be store in folder 'img' and 'label' under given opPath

  @param imgPath: folder containing the input nifti file
  @param gtPath: folder contatining the groud truth nifti file
  @param opPath: output folder
  @param normalize: normalize the input or not
  @param oneHot: convert the mask to one hot image or not
  @param newSize: the size of the ouput img/label
  @param cataNum: number of catagories for mask

  example:
  imgPath = './COVID-19-CT-Seg_20cases/'
  gtPath = './Lung_and_Infection_Mask/'
  savePath = './processed/'
  pre_processing(imgPath, gtPath, savePath)

  """
  imgFile = imgPath
  gtFile = gtPath
  savePath = opPath

  gtList = os.listdir(gtFile)

  opImgPath = os.path.join(savePath,'img/')
  opGtPath = os.path.join(savePath,'label/')


  if not os.path.isdir(savePath):
    os.mkdir(savePath)
  if not os.path.isdir(opImgPath):
    os.mkdir(opImgPath)
  if not os.path.isdir(opGtPath):
    os.mkdir(opGtPath)

  studyNum = len(gtList)

  for i in range(studyNum):
    gt = nib.load(gtFile + gtList[i]).get_fdata()
    img = nib.load(imgFile + gtList[i]).get_fdata()

    slices = img.shape[2]
    for s in range(slices):
      if oneHot:
        gtTemp = oneHotMask(gt[:,:,s], cataNum=cataNum)
      else:
        gtTemp = gt[:,:,s]
      gtTemp = resize(gtTemp, newSize, preserve_range=True)
      gtTemp = gtTemp.astype(np.uint8)
      imgTemp = resize(img[:,:,s], newSize, preserve_range=True)
      imgTemp = imgNorm(imgTemp)
      imgTemp = imgTemp.astype(np.uint8)
      gtName = 'case_' + str(i+1) + '_slices_' + str(s+1) + '.png'
      imgName = 'case_' + str(i+1) + '_slices_' + str(s+1) + '.png'
      imsave(opImgPath+imgName, imgTemp, check_contrast=False)
      imsave(opGtPath+gtName, gtTemp, check_contrast=False)

  return

## Download the Dataset

In [0]:
#Download the dataset
!wget https://zenodo.org/record/3757476/files/COVID-19-CT-Seg_20cases.zip?download=1
!wget https://zenodo.org/record/3757476/files/Lung_and_Infection_Mask.zip?download=1

--2020-06-10 07:14:13--  https://zenodo.org/record/3757476/files/COVID-19-CT-Seg_20cases.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1105395143 (1.0G) [application/octet-stream]
Saving to: ‘COVID-19-CT-Seg_20cases.zip?download=1’

COVID-19-CT-Seg_20c 100%[===================>]   1.03G  5.79MB/s    in 3m 54s  

2020-06-10 07:18:08 (4.51 MB/s) - ‘COVID-19-CT-Seg_20cases.zip?download=1’ saved [1105395143/1105395143]

--2020-06-10 07:18:10--  https://zenodo.org/record/3757476/files/Lung_and_Infection_Mask.zip?download=1
Resolving zenodo.org (zenodo.org)... 188.184.117.155
Connecting to zenodo.org (zenodo.org)|188.184.117.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11714838 (11M) [application/octet-stream]
Saving to: ‘Lung_and_Infection_Mask.zip?download=1’

Lung_and_Infection_ 100%[===================>]  11

## Rename the Files

In [0]:
#Rename zip file
!mv COVID-19-CT-Seg_20cases.zip?download=1 COVID-19-CT-Seg_20cases.zip
!mv Lung_and_Infection_Mask.zip?download=1 Lung_and_Infection_Mask.zip

## extract NIFTI data 

In [0]:
!mkdir images && unzip COVID-19-CT-Seg_20cases.zip -d images
!mkdir masks && unzip Lung_and_Infection_Mask.zip -d masks
!mkdir Dataset
!rm images/ReadMe.txt

Archive:  COVID-19-CT-Seg_20cases.zip
  inflating: images/coronacases_001.nii.gz  
  inflating: images/coronacases_002.nii.gz  
  inflating: images/coronacases_003.nii.gz  
  inflating: images/coronacases_004.nii.gz  
  inflating: images/coronacases_005.nii.gz  
  inflating: images/coronacases_006.nii.gz  
  inflating: images/coronacases_007.nii.gz  
  inflating: images/coronacases_008.nii.gz  
  inflating: images/coronacases_009.nii.gz  
  inflating: images/coronacases_010.nii.gz  
  inflating: images/radiopaedia_10_85902_1.nii.gz  
  inflating: images/radiopaedia_10_85902_3.nii.gz  
  inflating: images/radiopaedia_14_85914_0.nii.gz  
  inflating: images/radiopaedia_27_86410_0.nii.gz  
  inflating: images/radiopaedia_29_86490_1.nii.gz  
  inflating: images/radiopaedia_29_86491_1.nii.gz  
  inflating: images/radiopaedia_36_86526_0.nii.gz  
  inflating: images/radiopaedia_40_86625_0.nii.gz  
  inflating: images/radiopaedia_4_85506_1.nii.gz  
  inflating: images/radiopaedia_7_85703_0.nii

## PreProcess NIFTI to PNG

In [0]:
# shutil.rmtree('label', ignore_errors=True)
# os.mkdir('Dataset')

In [0]:
t1 = time.time()
pre_processing('images/','masks/','Dataset/')
t2 = time.time()
# Split Data into 
!mkdir('Dataset2')
sf.ratio('Dataset/', output='Dataset2/', seed=1337, ratio=(.7, .15, .15)) 
t3 = time.time()
print('PreProcessing Time is : ',t2-t1,'\n Splitting Time is : ',t3-t2)


/bin/bash: -c: line 0: syntax error near unexpected token `'Dataset2''
/bin/bash: -c: line 0: `mkdir('Dataset2')'


Copying files: 7040 files [00:01, 4232.56 files/s]

PreProcessing Time is :  385.77105593681335 
 Splitting Time is :  3.441913604736328


## Remove Unnecessary Files

In [0]:
shutil.rmtree('Dataset', ignore_errors=True)
shutil.rmtree('images', ignore_errors=True)
shutil.rmtree('masks', ignore_errors=True)
shutil.rmtree('sample_data', ignore_errors=True)
! rm COVID-19-CT-Seg_20cases.zip
! rm Lung_and_Infection_Mask.zip

## Data Generator

## Load Dataset (DataGenerator)

In [0]:
#--------------------------------------------------------------
# load your data
# Path
train_images_path = 'Dataset2/train/img/'
train_labels_path = 'Dataset2/train/label/'

val_images_path   = 'Dataset2/val/img/'
val_labels_path   = 'Dataset2/val/label/'

test_images_path  = 'Dataset2/test/img/'
test_labels_path  = 'Dataset2/test/label/'
#--------------------------------------------------------------
train_generator      = DataGenerator(train_images_path, train_labels_path, image_dimensions=(256,256),batch_size=2)
val_generator        = DataGenerator(val_images_path, val_labels_path, image_dimensions=(256,256),batch_size=2)
test_generator_eval  = DataGenerator(test_images_path, test_labels_path, image_dimensions=(256,256),batch_size=2)
test_generator       = DataGenerator(test_images_path, test_labels_path, mode='test',image_dimensions=(256,256),batch_size=2)

## Fitting(Training)

In [0]:
# Model Fitting (Training)
#=========================
batch_size = 8
# Model Checkpoint
Callbacks = [
             keras.callbacks.EarlyStopping(patience=5,monitor='val_loss')
             ,keras.callbacks.ModelCheckpoint('DIUNet_Weights.h5',
             verbose=1, save_best_only=True)]

# Fitting (Training)             
history = model.fit_generator(generator=train_generator,
    validation_data=val_generator,epochs=120, 
    steps_per_epoch= 300, # len(os.listdir(train_images_path)) // batch_size
    validation_steps = 300, # len(os.listdir(val_images_path))// batch_size
    verbose=1,
    callbacks = Callbacks)

#=================
## Save Results
#=================
# Copy the Model to The Drive
!cp DIUNet_Weights.h5 drive/'My Drive'/Dataset/Covid_19/

# #==============
# # Save History
# #==============
np.save('drive/My Drive/Dataset/Covid_19/DIUNet_history_ES.npy',history.history)
np.save('DIUNet_history_ES.npy',history.history)
# #================
# # Save Model
# #================
# model.save('drive/My Drive/Dataset/Covid_19/DIUNet_Model.h5')
# Delete Model
# del model
# del history

Epoch 1/120


## Evaluate The Model

In [0]:
#================
# Evaluate Model
#================
score = model.evaluate(test_generator_eval)
print('Test loss:', score[0])
print('Test accuracy', score[1])

## Visualize the Metrics

In [0]:
#=================================
"""### Visualize fitting curve""" 
#================================= 
plt.figure(figsize=(30, 5), num = 'Metrics')
plt.subplot(121)
plt.plot(history.history['iou_score'])
plt.plot(history.history['val_iou_score'])
plt.title('Model iou_score')
plt.ylabel('iou_score')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(122)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')


In [0]:
#================
# Predict Model
#================
test_images, test_labels = next(iter(test_generator_eval))
prediction = model.predict(test_generator)
#-----------------------------------------
prediction[prediction>=0.5]=1
prediction[prediction<0.5]=0
pred = np.squeeze(prediction)
print('pred shape: ',pred.shape)
# pred = np.squeeze(np.sum(prediction,axis=-1))
# pred = 255*(pred-pred.min())/(pred.max()-pred.min())

masks = np.squeeze(test_labels)
# masks = np.squeeze(np.sum(test_labels,axis=-1))
# mask = 255*(mask-mask.min())/(mask.max()-mask.min())
print('masks shape: ',masks.shape)

images = np.squeeze(test_images)
print('images shape: ',images.shape)

## Visualize Prediction

In [0]:
#===================================
"""### Visualize the Results"""
#===================================
print('pred shape: ',pred.shape,'masks.shape: ',masks.shape,'images.shape: ',images.shape,'\n' )
plt.figure(num='LinkNet_EB7')

plt.subplot('221')
plt.imshow(images[0],cmap='gray')
plt.title('Train Image')
plt.axis('off')

plt.subplot('222')
plt.imshow(masks[0])
plt.title('Train Mask')
plt.axis('off')

plt.subplot('223')
plt.imshow(images[1],cmap='gray')
plt.title('Test Image')
plt.axis('off')

plt.subplot('224')
plt.imshow(pred[1])
plt.title('Prediction')
plt.axis('off')

plt.show()